# Callbacks and Multiple inputs

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale

from keras.optimizers import SGD
from keras.layers import Dense, Input, concatenate, BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.models import Model
import keras.backend as K

In [ ]:
df = pd.read_csv("../data/titanic-train.csv")
Y = df['Survived']

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
num_features = df[['Age', 'Fare', 'SibSp', 'Parch']].fillna(0)
num_features.head()

In [ ]:
cat_features = pd.get_dummies(df[['Pclass', 'Sex', 'Embarked']].astype('str'))
cat_features.head()

In [ ]:
X1 = scale(num_features.values)
X2 = cat_features.values

In [ ]:
K.clear_session()

# Numerical features branch
inputs1 = Input(shape = (X1.shape[1],))
b1 = BatchNormalization()(inputs1)
b1 = Dense(3, kernel_initializer='normal', activation = 'tanh')(b1)
b1 = BatchNormalization()(b1)

# Categorical features branch
inputs2 = Input(shape = (X2.shape[1],))
b2 = Dense(8, kernel_initializer='normal', activation = 'relu')(inputs2)
b2 = BatchNormalization()(b2)
b2 = Dense(4, kernel_initializer='normal', activation = 'relu')(b2)
b2 = BatchNormalization()(b2)
b2 = Dense(2, kernel_initializer='normal', activation = 'relu')(b2)
b2 = BatchNormalization()(b2)

merged = concatenate([b1, b2])
preds = Dense(1, activation = 'sigmoid')(merged)

# final model
model = Model([inputs1, inputs2], preds)

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

model.summary()

In [ ]:
outpath='/tmp/tensorflow_logs/titanic/'

early_stopper = EarlyStopping(monitor='val_acc', patience=10)
tensorboard = TensorBoard(outpath+'tensorboard/', histogram_freq=1)
checkpointer = ModelCheckpoint(outpath+'weights_epoch_{epoch:02d}_val_acc_{val_acc:.2f}.hdf5',
                               monitor='val_acc')

In [ ]:
# You may have to run this a couple of times if stuck on local minimum
np.random.seed(2017)
h = model.fit([X1, X2],
              Y.values,
              batch_size = 32,
              epochs = 40,
              verbose = 1,
              validation_split=0.2,
              callbacks=[early_stopper,
                         tensorboard,
                         checkpointer])

In [ ]:
import os
sorted(os.listdir(outpath))

Now check the tensorboard.

- If using provided aws instance, just browse to: `http://<your-ip>:6006`

- If using local, open a terminal, activate the environment and run:
```
tensorboard --logdir=/tmp/tensorflow_logs/titanic/tensorboard/
```
then open a browser at `localhost:6006`

You should see something like this:

![tensorboard.png](../assets/tensorboard.png)

## Exercise 1

- try modifying the parameters of the 3 callbacks provided. What are they for? What do they do?

*Copyright &copy; 2017 Francesco Mosconi & CATALIT LLC. All rights reserved.*